In [11]:
import os
import requests
import pandas as pd
import datetime

In [ ]:
today_date = datetime.datetime.now().date()
yesterday_date = today_date - datetime.timedelta(days=1)
two_days_ago_date = today_date - datetime.timedelta(days=2)
print(yesterday_date)
print(two_days_ago_date)

In [13]:
ALPHA_API_KEY = os.environ.get('ALPHA_API_KEY')
STOCK = "AAPL"

alpha_url = 'https://www.alphavantage.co/query?'
alpha_params = {
    'function': 'TIME_SERIES_DAILY',
    'symbol': STOCK,
    'apikey': ALPHA_API_KEY
}

In [14]:
def get_stock_variation(url, params) -> bool:
    response = requests.get(url, params=params)
    response.raise_for_status()
    raw_data = response.json()
    data = raw_data["Time Series (Daily)"]
    
    # Get sorted available dates from API
    dates = sorted(data.keys(), reverse=True)
    
    try:
        yesterday_date = dates[0]
        two_days_ago_date = dates[1]
    except IndexError:
        print("Not enough data to calculate stock variation")
        return False
    
    # Fetch close prices
    yesterday_close = float(data[yesterday_date]["4. close"])
    two_days_close = float(data[two_days_ago_date]["4. close"])
    
    percentage_difference = (yesterday_close - two_days_close) / yesterday_close * 100
    
    if abs(percentage_difference) >= 5:
        print("The variation is more than 5%")
        return True
    else:
        print("The variation is within 5%")
        return False

In [15]:
news_url = 'https://newsapi.org/v2/everything'
news_params = {
    'apiKey': os.environ.get('NEWS_API_KEY'),
    'q': 'apple',
    'from': yesterday_date,
    'language': 'en',
    'sortBy':'relevancy'
}

In [16]:
def get_news(url, params) -> list:
    response = requests.get(url, params)
    response.raise_for_status()
    data = response.json() # o: python dict
    # print(data)
    # print(type(data))

    return data['articles'][0:3]

In [ ]:
# TEST
stock_price_variation = get_stock_variation(alpha_url, alpha_params)
print(stock_price_variation)
# news = get_news(news_url, news_params)
# print(news)